# About

It's difficult to use functions in jupyter notebook, since we want different steps to be in different cells, so one of the main functions of this module is to emulate a function like scope of the variables - which get destroyed at the end of the experiment. Some extra magic is added to reclaim GPU and General RAM.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from ipyexperiments import *

## Setup and preload

In [3]:
import numpy as np
import torch

In [4]:
def consume_gpu(n): return torch.ones((n, n)).cuda()
def consume_cpu(n): return np.ones((n, n))

`pytorch`'s CUDA machinery seems to require ~0.5GB GPU RAM, and ~2GB of RAM upon its first use, and it's not shared between processes. So if you use pytorch w/ CUDA your GPU card is 0.5GB smaller from the get going, and multiply that by the number of concurrent processes. 

Because of that, in order to get the numbers right, it can be a good idea to pre-load it by allocating a tiny tensor on `cuda`. If we don't - the first experiment' stats will be misleading/incorrect.

In [5]:
#z = consume_gpu(1)

But, if you use `IPyExperiments` - it performs this preloading for you already, when the backend is loaded (see below).

## Experiment with no GPU

Let's consume a big chunk of non-GPU RAM and reclaim it at the end of the experiment.

In this experiment we use the `cpu` backend, so GPU RAM will not be managed, regardless of whether there is a GPU that can be used or not. This mode is primarily used for configurations without GPU.

In [6]:
exp1 = IPyExperimentsCPU() # consume some cpu ram


*** Starting experiment...
Backend: CPU-only

*** Current state:
RAM:   Used      Free     Total    Util
CPU: 165.6 MB   17.2 GB  17.4 GB   0.94% 




In [7]:
x1 = consume_cpu(2**14) # about 2GB

In [8]:
x2 = consume_cpu(2**14) # about 2GB

In [9]:
del exp1 # finish experiment


*** Finishing experiment...

*** Deleting the following local variables:
['x1', 'x2']

*** RAM consumed during the experiment:
CPU: 4.0 GB

*** RAM reclaimed at the end of the experiment:
CPU: 4.0 GB (100.00%)

*** Elapsed wallclock time:
00:00:01

*** Current state:
RAM:   Used      Free     Total    Util
CPU: 165.7 MB   17.2 GB  17.4 GB   0.94% 




## GPU Experiment: consume general and GPU RAM

Let's consume a big chunk of each, general and GPU RAM and reclaim both of them, at the end of the experiment.

This time we wil use the GPU backed `pytorch`, so both GPU and general RAM will be managed. This is the default backed, so if you don't pass this argument, it'll default to `pytorch`.

In [10]:
exp2 = IPyExperimentsPytorch()


*** Starting experiment...
Backend: Pytorch
Device: ID 0, GeForce GTX 1070 Ti (7.9 GB RAM)

*** Current state:
RAM:   Used      Free     Total    Util
CPU:   2.1 GB   15.5 GB  17.6 GB  13.59% 
GPU:   1.6 GB    6.3 GB   7.9 GB  25.83% 




In [11]:
x1 = consume_cpu(2**14) # about 2GB

In [12]:
x2 = consume_gpu(2**14) # about 1GB

In [13]:
del exp2 # finish experiment


*** Finishing experiment...

*** Deleting the following local variables:
['x1', 'x2']

*** RAM consumed during the experiment:
CPU: 2.0 GB
GPU: 1.0 GB

*** RAM reclaimed at the end of the experiment:
CPU: 2.0 GB (99.98%)
GPU: 1.0 GB (100.00%)

*** Elapsed wallclock time:
00:00:02

*** Current state:
RAM:   Used      Free     Total    Util
CPU:   2.1 GB   15.5 GB  17.6 GB  13.60% 
GPU:   1.6 GB    6.3 GB   7.9 GB  25.83% 




## Get experiment data, preserve some vars

Here we demonstate features that help with using this framework programmatically. i.e. getting the functions to return experiment data during and at the end of the experiment, rather than just printing it. You can then use it to programmatically refine the hyper parameters before rerunning the experiment.

This experiment also demonstrates how to save some of the local variables.

In [14]:
exp3 = IPyExperimentsPytorch() # consume some gpu and cpu ram


*** Starting experiment...
Backend: Pytorch
Device: ID 0, GeForce GTX 1070 Ti (7.9 GB RAM)

*** Current state:
RAM:   Used      Free     Total    Util
CPU:   2.1 GB   15.5 GB  17.6 GB  13.60% 
GPU:   1.6 GB    6.3 GB   7.9 GB  25.83% 




In [15]:
x1 = consume_cpu(2**14) # about 1.5GB

Run an intermediary report of how much of the resources was consumed, and how much is available, returning the data as numbers. (none would be reclaimed yet, so it'll be zeros, but the return value is there for consistency).

In [16]:
cpu_data, gpu_data = exp3.data
print(cpu_data, gpu_data)

IPyExperimentMemory(consumed=2147274752, reclaimed=0, available=14486392832) IPyExperimentMemory(consumed=0, reclaimed=0, available=6766002176)


Let's preserve these variables, so that they remain available after the experiment is finished and the rest of the local variables get deleted. 

Note, that you need to pass the names of the variables and not the variables themselves.

In [17]:
exp3.keep_var_names('cpu_data', 'gpu_data')

In [18]:
x2 = consume_gpu(2**14) # about 1GB

Run another intermediary report.

In [ ]:
cpu_data, gpu_data = exp3.data
print(cpu_data, gpu_data)
print(cpu_data.consumed)
print(gpu_data.available)

IPyExperimentMemory(consumed=2147504128, reclaimed=0, available=14483275776) IPyExperimentMemory(consumed=1073741824, reclaimed=0, available=5692260352)
2147504128
5692260352


Complete the experiment, delete local vars, reclaim memory, and run the final report of how much of the resources was consumed, and how much is available, and how much was reclaimed, returning the data as numbers.

In [ ]:
cpu_data_final, gpu_data_final = exp3.finish() # finish experiment
print("\nNumerical data:\n", cpu_data_final, gpu_data_final)


*** Finishing experiment...

*** Deleting the following local variables:
['exp3', 'x1', 'x2']

*** Keeping the following local variables:
['cpu_data', 'gpu_data']

*** RAM consumed during the experiment:
CPU: 2.0 GB
GPU: 1.0 GB

*** RAM reclaimed at the end of the experiment:
CPU: 2.0 GB (100.00%)
GPU: 1.0 GB (100.00%)

*** Elapsed wallclock time:
00:00:01

*** Current state:
RAM:   Used      Free     Total    Util
CPU:   2.1 GB   15.5 GB  17.6 GB  13.61% 
GPU:   1.6 GB    6.3 GB   7.9 GB  25.83% 



Numerical data:
 IPyExperimentMemory(consumed=2147504128, reclaimed=2147487744, available=16632782848) IPyExperimentMemory(consumed=1073741824, reclaimed=1073741824, available=6766002176)


And let's test that we can still access the variables we asked not to destroy:

In [ ]:
print("\nHalf-way data:\n", cpu_data, gpu_data)


Half-way data:
 IPyExperimentMemory(consumed=2147504128, reclaimed=0, available=14483275776) IPyExperimentMemory(consumed=1073741824, reclaimed=0, available=5692260352)


## Using the context manager

If you want to put all cells into one, you could simplify the experiment even further by using its context manager.

In [ ]:
with IPyExperimentsPytorch(): 
    x1 = consume_cpu(2**14)
    x2 = consume_gpu(2**14)


*** Starting experiment...
Backend: Pytorch
Device: ID 0, GeForce GTX 1070 Ti (7.9 GB RAM)

*** Current state:
RAM:   Used      Free     Total    Util
CPU:   2.1 GB   15.5 GB  17.6 GB  13.60% 
GPU:   1.6 GB    6.3 GB   7.9 GB  25.83% 



*** Finishing experiment...

*** Deleting the following local variables:
['x1', 'x2']

*** RAM consumed during the experiment:
CPU: 2.0 GB
GPU: 1.0 GB

*** RAM reclaimed at the end of the experiment:
CPU: 2.0 GB (100.00%)
GPU: 1.0 GB (100.00%)

*** Elapsed wallclock time:
00:00:00

*** Current state:
RAM:   Used      Free     Total    Util
CPU:   2.1 GB   15.5 GB  17.6 GB  13.61% 
GPU:   1.6 GB    6.3 GB   7.9 GB  25.83% 




In [ ]:
with IPyExperimentsPytorch() as exp: 
    x1 = consume_cpu(2**14)
    z = "some data"
    x2 = consume_gpu(2**14)
    exp.keep_var_names('z')
print(z)


*** Starting experiment...
Backend: Pytorch
Device: ID 0, GeForce GTX 1070 Ti (7.9 GB RAM)

*** Current state:
RAM:   Used      Free     Total    Util
CPU:   2.1 GB   15.5 GB  17.6 GB  13.60% 
GPU:   1.6 GB    6.3 GB   7.9 GB  25.83% 



*** Finishing experiment...

*** Deleting the following local variables:
['exp', 'x1', 'x2']

*** Keeping the following local variables:
['z']

*** RAM consumed during the experiment:
CPU: 2.0 GB
GPU: 1.0 GB

*** RAM reclaimed at the end of the experiment:
CPU: 2.0 GB (100.00%)
GPU: 1.0 GB (100.00%)

*** Elapsed wallclock time:
00:00:00

*** Current state:
RAM:   Used      Free     Total    Util
CPU:   2.1 GB   15.5 GB  17.6 GB  13.61% 
GPU:   1.6 GB    6.3 GB   7.9 GB  25.83% 


some data


In [ ]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()